In [167]:
import tensorflow as tf
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [175]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.01, shape=shape))

In [176]:
def conv2d(x, W, stride_size=2):
    return tf.nn.conv2d(x, W, strides=[1,stride_size,stride_size,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [213]:
WIDTH = 640 # pixel width of each image
HEIGHT = 640 # pixel height of each image

x = tf.placeholder(tf.float32, shape=[None, WIDTH*HEIGHT]) # input is flattened image
y = tf.placeholder(tf.float32, shape=[None, 1]) # output is one value

x_image = tf.reshape(x, [-1, WIDTH, HEIGHT, 1])

# Weights and biases between input and hidden layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Hidden layer 1
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, stride_size=4) + b_conv1)
# Max pooled hidden layer 1
h_pool1 = max_pool_2x2(h_conv1) #dim is 80x80x32

print('h_pool1', h_pool1.shape)

# Weights and biases between hidden layer 1 and 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = weight_variable([64])

# Hidden layer 2
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, stride_size=4) + b_conv2)
# Max pooled hidden layer 2
h_pool2 = max_pool_2x2(h_conv2) # dim is 20x20x64

print('h_pool2', h_pool2.shape)

# Weights and biases between hidden layer 2 and 3
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

# Hidden layer 3
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
# Max pooled hidden layer 3
h_pool3 = max_pool_2x2(h_conv3) # dim is 5x5x128

print('h_pool3', h_pool3.shape)

h_pool1 (?, 80, 80, 32)
h_pool2 (?, 10, 10, 64)
h_pool3 (?, 3, 3, 128)


In [215]:
# Dense layer 1
W_d1 = weight_variable([3*3*128, 512])
b_d1 = weight_variable([512])

h_pool3_flat = tf.reshape(h_pool3, [-1, 3*3*128])
h_d1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_d1) + b_d1)

keep_prob = tf.placeholder(tf.float32)
h_d1_drop = tf.nn.dropout(h_d1, keep_prob)

# Dense layer 2
W_d2 = weight_variable([512, 1])
b_d2 = bias_variable([1])

# Output
y_pred = tf.matmul(h_d1_drop, W_d2) + b_d2

In [216]:
im = Image.open('images/0_61.png', 'r').convert('L')
pix = im.load()
im_array = []
for i in range(im.size[0]):
    for j in range(im.size[1]):
        im_array.append(pix[i,j])
im_array = np.expand_dims(np.array(im_array), axis=0) / 255

In [253]:
mse = tf.losses.mean_squared_error(y, y_pred)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(mse)
init = tf.global_variables_initializer()

In [258]:
with tf.Session() as sess:
    sess.run(init)
#     print(sess.run([y_pred], feed_dict={
#         x: im_array,
#         y: np.array([[105.0]]),
#         keep_prob: 0.5
#     }))
#     print(sess.run([y_pred], feed_dict={
#         x: im_array,
#         y: np.array([[105.0]]),
#         keep_prob: 1
#     }))
    for n in range(10):
        sess.run(train_step, feed_dict={
            x: im_array,
            y: np.array([[105]]),
            keep_prob: 0.5
        })
        print(sess.run([mse], feed_dict={
            x: im_array,
            y: np.array([[105]]),
            keep_prob: 1
        }))

[1.4643651e+23]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
